# Ссылка на датасет на kaggle.com
https://www.kaggle.com/datasets/ismetsemedov/transactions/data

In [156]:
import pandas as pd

### 1. Загрузить данные

In [157]:
data = pd.read_csv('partial_fraud_data.csv')

### 2a. Объем памяти, который занимает файл на диске

In [158]:
import os

file_size = os.path.getsize('partial_fraud_data.csv')
with open('stats/output.txt', 'w', encoding='utf-8') as f:
    f.write(f"2a. Объем памяти файла на диске: {file_size / 1024:.2f} КБ ({file_size / (1024 * 1024):.2f} МБ)\n")

### 2b. Объем памяти, который занимает набор данных при загрузке в память

In [159]:
with open('stats/output.txt', 'a', encoding='utf-8') as f:
    f.writelines(f"2b. Объем памяти на диске: {sum(data.memory_usage(deep=True)) // 1024:.2f} КБ\n")

### 2c. Для каждой колонки: объем памяти, доля от общей, тип данных

In [160]:
column_memory = data.memory_usage(deep=True)
total_memory = column_memory.sum()

with open('stats/output.txt', 'a', encoding='utf-8') as f:
    f.write('2c. По колонкам:\n')
    for col in data.columns:
        col_memory = column_memory[col]
        col_type = data[col].dtype
        mem_percentage = (col_memory / total_memory) * 100
        f.write(f"- Столбец: {col}\n")
        f.write(f"- Объем памяти: {col_memory / 1024:.2f} КБ\n")
        f.write(f"- Тип данных: {col_type}\n")
        f.write(f"- Доля от общего объема памяти: {mem_percentage:.2f}%\n\n")

### 3. Сортировка полученного набора по занимаемому объему памяти

In [ ]:
import json

columns_stat = []

for col in data.columns:
    
    col_memory = column_memory[col]
    col_type = data[col].dtype
    mem_percentage = (col_memory / total_memory) * 100
    
    columns_stat.append({
        "column": col,
        "memory_kb": round(col_memory / 1024, 2),
        "mem_percentage": round(mem_percentage, 2),
        "col_type": str(col_type),
        "note": "Statistics w/o optimization"
    })

columns_stat_sorted = sorted(columns_stat, key=lambda x: x['memory_kb'], reverse=True)

with open('stats/columns_stat.json', 'w', encoding='utf-8') as f:
    json.dump(columns_stat_sorted, f, ensure_ascii=False, indent=4)

### 4. Приобразование по object, уникальность < 50%

In [162]:
for col in data.select_dtypes(include=["object"]).columns:
    unique_val = data[col].nunique()
    unique_ratio = unique_val / len(data)

    if unique_ratio < 0.5:
        data[col] = data[col].astype("category")

### 5. Понижающее преобразование по типу int

In [163]:
for col in data.select_dtypes(include=["int64"]).columns:
    data[col] = pd.to_numeric(data[col], downcast="integer")

### 6. Понижающее преобразование по типу float

In [164]:
for col in data.select_dtypes(include=["float64"]).columns:
    data[col] = pd.to_numeric(data[col], downcast="float")

In [ ]:
data.dtypes

### 7. Повторный анализ

#### Объем памяти на диске

In [166]:
import os

file_size = os.path.getsize('new_fraud_data.csv')
with open('stats/output_after_optimization.txt', 'w', encoding='utf-8') as f:
    f.write(f"2a. Объем памяти файла на диске: {file_size / 1024:.2f} КБ ({file_size / (1024 * 1024):.2f} МБ)\n")

In [167]:
with open('stats/output_after_optimization.txt', 'a', encoding='utf-8') as f:
    f.write(f"2b. Объем памяти на диске: {sum(data.memory_usage(deep=True)) // 1024:.2f} КБ\n")

In [168]:
column_memory = data.memory_usage(deep=True)
total_memory = column_memory.sum()

with open('stats/output_after_optimization.txt', 'a', encoding='utf-8') as f:
    f.write('2c. По колонкам:\n')
    for col in data.columns:
        col_memory = column_memory[col]
        col_type = data[col].dtype
        mem_percentage = (col_memory / total_memory) * 100
        f.write(f"- Столбец: {col}\n")
        f.write(f"- Объем памяти: {col_memory / 1024:.2f} КБ\n")
        f.write(f"- Тип данных: {col_type}\n")
        f.write(f"- Доля от общего объема памяти: {mem_percentage:.2f}%\n\n")